In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import json

In [2]:
# 알라딘 OpenAPI KEY
with open("../data/API_KEYS.json") as f :
    api_key = json.loads(f.read())
key = api_key["ALADIN_KEY"]

In [3]:
# 2020-2023 역대 베스트셀러 itemId 추출
itemId_lst = []
for year in range(2020, 2024) :
    for i in range(1, 7) :
        url = f"https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=YearlyBest&BranchType=1&CID=0&Year={year}&page={i}&cnt=300&SortOrder=1"
        res = req.get(url)
        soup = bs(res.content, "html.parser")
        items = soup.select("div.ss_book_box")
        for i in items :
            if "itemid" in i.attrs :
                itemId_lst.append(i["itemid"])

In [4]:
# 중복 값 제거
itemId_lst = set(itemId_lst)
itemId_lst = list(itemId_lst)
len(itemId_lst)

1006

In [7]:
# JSON 변환
books = []
for i in tqdm(range(len(itemId_lst))) :
    # 도서 상세 정보
    url = f"http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={key}&itemIdType=ItemId&ItemId={itemId_lst[i]}&output=js&Version=20131101"
    res = req.get(url, timeout = 15)
    # 응답 받은 JSON 데이터 변수에 할당
    data = res.json()
    books.extend(data["item"])

# JSON 데이터를 파일로 저장
with open("../data/book_info.json", "w", encoding = "utf-8") as f :
    json.dump(books, f, ensure_ascii = False, indent = 4)
print("완료😇")

100%|██████████| 1006/1006 [01:39<00:00, 10.12it/s]

완료😇
